In [1]:
cd ~/microbiome_rnn/

/Volumes/export/isn/mwiest/microbiome_rnn


In [14]:
import torch
import os
import sys
import torch.nn as nn
import pandas as pd
from otu_handler import OTUHandler

In [4]:
input_dir = 'all_studies_reduced/'
files = []
for (dirpath, dirnames, filenames) in os.walk(input_dir):
    files.extend(filenames)
    break
files = [os.path.join(input_dir, f) for f in files if not f.endswith('_clr.csv')]

# Generate the data handler object
otu_handler = OTUHandler(files)


In [10]:
otu_handler.samples[3]

,0,1,2,3,4,5,6,7,8,9,...,326,327,328,329,330,331,332,333,334,335
k__Bacteria; p__Firmicutes; c__Bacilli; o__Lactobacillales; f__Enterococcaceae; g__Enterococcus; s__,5.139702,4.185938,4.678913,6.269117,3.677922,2.976013,5.792167,6.733125,6.617822,2.494497,...,7.393407,7.291650,6.401194,7.163030,7.907646,7.220550,2.060682,8.418469,7.326475,8.066193
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Clostridiaceae,4.243916,3.468458,3.715619,4.491471,2.485761,2.321333,3.638298,4.337459,4.381371,1.924756,...,4.826831,4.632425,3.915178,4.277141,4.860904,4.092843,1.207884,5.059712,4.757158,4.924020
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Lachnospiraceae,6.885333,5.421366,8.548089,18.964512,13.428317,6.084506,25.410889,27.871466,25.571134,5.423769,...,29.617182,31.183494,29.712766,34.853186,36.330047,38.505586,10.374920,40.453829,29.749742,37.588201
k__Bacteria; p__Firmicutes; c__Erysipelotrichi; o__Erysipelotrichales; f__Erysipelotrichaceae; g__Bulleidia; s__,10.337277,8.465593,8.840320,9.966446,5.271275,5.460937,7.235286,8.827693,9.132426,4.496405,...,9.940329,9.309876,7.588421,8.090386,9.468128,7.311955,2.240377,9.635388,9.750139,9.525484
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Lachnospiraceae; g__[Ruminococcus]; s__,20.820119,17.039571,17.937624,20.687264,11.110807,11.120677,15.598231,18.875206,19.364260,9.177120,...,21.166191,19.995351,16.512054,17.762466,20.565209,16.402417,4.954793,21.101960,20.796046,20.741500
k__Bacteria; p__Fusobacteria; c__Fusobacteriia; o__Fusobacteriales; f__Fusobacteriaceae; g__Fusobacterium; s__,852.031397,698.001570,725.705757,807.834212,423.507298,447.401453,573.582006,703.285052,731.170631,367.921886,...,793.881024,739.720364,598.191008,634.249240,747.188761,565.475720,174.835256,756.537376,777.918201,750.566997
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Lachnospiraceae; g__Anaerostipes; s__,9.504157,7.762325,8.384827,10.354116,5.804610,5.257253,8.641650,10.240974,10.279288,4.368689,...,11.361001,10.973260,9.359228,10.285080,11.605812,9.972360,2.918023,12.147251,11.211185,11.776412
k__Bacteria; p__Firmicutes; c__Bacilli; o__Lactobacillales; f__Streptococcaceae; g__Lactococcus; s__,5.104018,4.177099,4.398887,5.078437,2.729419,2.727603,3.835556,4.639694,4.758166,2.251133,...,5.201895,4.915992,4.061884,4.371145,5.058550,4.040096,1.219696,5.192408,5.111302,5.102459
k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__,12484.604676,10240.773232,10472.876376,11092.232596,5606.333563,6407.802972,7160.381271,8976.302728,9536.155723,5244.313896,...,10243.082516,9329.469066,7275.258297,7513.221971,9134.309877,6253.490899,2025.116273,9028.860485,9993.519673,9110.099453
k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Lachnospiraceae; g__Lachnospira; s__,195.209304,159.420228,172.377620,213.402099,119.814468,108.126565,178.722115,211.654703,212.292353,89.874937,...,234.719266,226.873783,193.703273,213.006335,240.166673,206.832025,60.464128,251.526790,231.657608,243.743446


In [103]:
r = torch.FloatTensor(torch.randn(50, 15, 100))
hidden_dim = 128

In [110]:
conv_element = nn.Sequential(
            nn.Conv1d(100, 64,
                      kernel_size=4, stride=2, padding=3),
            nn.ReLU(),
            nn.Conv1d(64, 128,
                      kernel_size=2, stride=1, padding=1),

            nn.ReLU(),
            nn.Conv1d(128, 128,
                  kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(128, 128,
                      kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(128, 128,
                      kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(128, 100,
                      kernel_size=1, stride=1, padding=0),
            nn.ReLU(),
        )

deconv_element = nn.Sequential(
    nn.ConvTranspose1d(512, 128, kernel_size=1, stride=1, padding=0),
    nn.ReLU(),
    nn.ConvTranspose1d(128, 128, kernel_size=2, stride=1, padding=1),
        nn.ReLU(),
    nn.ConvTranspose1d(128, 128, kernel_size=2, stride=1, padding=1),
        nn.ReLU(),
        nn.ConvTranspose1d(128, 128, kernel_size=2, stride=1, padding=1),
        nn.ReLU(),
        nn.ConvTranspose1d(128, 64, kernel_size=2, stride=1, padding=1),
        nn.ReLU(),
    nn.ConvTranspose1d(64, 100, kernel_size=4, stride=2, padding=3),
        nn.ReLU()
)
    

before_lstm = nn.Sequential(
            nn.Linear(100, hidden_dim),
            # nn.Linear(hidden_dim, hidden_dim),
            # nn.BatchNorm1d(hidden_dim),
            nn.Tanh(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, 512),
            # nn.BatchNorm1d(self.otu_handler.num_strains)
            nn.Tanh()
        )
mp = nn.MaxPool1d(kernel_size=2, stride=2)

In [111]:
print(r.size())
c = conv_element(r.transpose(1,2))
print(c.size())
b = before_lstm(c.transpose(1,2))
print(b.size())
d = deconv_element(b.transpose(1,2))
print(d.size())

torch.Size([50, 15, 100])
torch.Size([50, 100, 13])
torch.Size([50, 13, 512])
torch.Size([50, 100, 14])


In [53]:
before_lstm(r).size()

torch.Size([50, 15, 20])